In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pickle
import os
from tqdm import tqdm_notebook as tqdm
#from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

In [ ]:
class Note():
    def __init__(self, text, index, source_text_length):
        self.index = index
        self.text = text
        self.source_text_length = source_text_length
        self.lines_to_check = []
        self.lines = []
        
        self.previous = None
        self.next = None
        
        self.done = False
        
    def __str__(self):
        return '(Note {}) {}'.format(self.index, self.text[:50].strip())
    
    def get_search_start(self):
        if self.previous is None:
            return 0
        elif self.previous.lines != []:
            return self.previous.lines[-1]
        else:
            return self.previous.get_search_start()
        
    def get_search_end(self):
        if self.next is None:
            return self.source_text_length
        elif self.next.lines != []:
            return self.next.lines[0]
        else:
            return self.next.get_search_end()
            
    def get_search_indices(self):
        return range((self.get_search_start()+ 1), self.get_search_end())
            
    def to_fragment(self, lines):
        return ''.join([lines[i] for i in self.lines])
    
    def ed(self, lines):
        fragment = self.to_fragment(lines)
        return edlib.align(self.text, fragment)['editDistance']
    
    def complete(self, lines, threshold=0.2):
        ed = self.ed(lines)
        score = ed/len(self.text)
        if score < threshold:
            self.done = True
        return self.done


In [ ]:
pickle_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/pickled'

fname = '_aio001jver01_01.txt'

with open(os.path.join(pickle_dir, fname),'rb') as f:
     ns = pickle.load(f)

In [ ]:
for n in ns:
    print(n, n.found, n.repeated, n.lines)